# MCP Network Demo (Streamable HTTP Transport)

This notebook demonstrates connecting to a **remote MCP server** over the network using Streamable HTTP, in contrast to the stdio transport in d2-mcp-demo.

We'll connect to CoinGecko's public MCP server (no API key required).

See https://docs.coingecko.com/docs/mcp-server


In [ ]:
import json
import logging
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

logging.getLogger("mcp.client.streamable_http").setLevel(logging.ERROR)

MCP_SERVER_URL = "https://mcp.api.coingecko.com/mcp"

In [ ]:
async def list_remote_tools():
    """Connect to remote MCP server and list available tools"""
    async with streamablehttp_client(url=MCP_SERVER_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await session.list_tools()
            return tools

tools_response = await list_remote_tools()

print(f"Found {len(tools_response.tools)} tools:\n")
for tool in tools_response.tools:
    print(f"- {tool.name}")
    print(f"  {tool.description}\n")

In [ ]:
async def call_remote_tool(tool_name: str, args: dict):
    """Execute a tool on the remote MCP server"""
    async with streamablehttp_client(url=MCP_SERVER_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, arguments=args)
            return result

# Get Bitcoin price
result = await call_remote_tool("get_simple_price", {"ids": "bitcoin", "vs_currencies": "usd"})

for content in result.content:
    if hasattr(content, 'text'):
        data = json.loads(content.text)
        print(json.dumps(data, indent=2))

## Agentic Pattern: LLM + MCP Tools

The previous cells called MCP tools directly with hardcoded parameters. Now we'll let **Claude** decide which tools to call and how to interpret the results.

This is the core agentic pattern:
1. Send a question and available tools to Claude
2. Claude decides which tool(s) to call and with what parameters
3. We execute the tool calls via MCP and feed results back
4. Claude reasons over the data and provides a final answer

In [ ]:
import os
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv(override=True)

anthropic_client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

# Convert MCP tools to Claude's tool format
claude_tools = []
for tool in tools_response.tools:
    claude_tools.append({
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    })

print(f"Registered {len(claude_tools)} tools for Claude")

claude_tools

## Ask Claude: BTC vs ETH in the Last 24 Hours

We'll ask Claude to compare BTC and ETH performance. Claude will autonomously:
- Determine which MCP tool(s) to call
- Choose the right parameters (e.g. requesting 24h change data)
- Analyze the results and provide a reasoned answer

In [ ]:
async def ask_claude_with_mcp(question: str):
    """Send a question to Claude, letting it use MCP tools to gather data."""
    print(f"Question: {question}\n")

    messages = [{"role": "user", "content": question}]

    response = anthropic_client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        tools=claude_tools,
        messages=messages
    )

    # Agentic loop: keep going while Claude wants to use tools
    while response.stop_reason == "tool_use":
        tool_use_blocks = [block for block in response.content if block.type == "tool_use"]

        if not tool_use_blocks:
            break

        messages.append({"role": "assistant", "content": response.content})

        # Execute each tool call via the remote MCP server
        tool_results = []
        for tool_block in tool_use_blocks:
            tool_name = tool_block.name
            tool_args = tool_block.input

            print(f"  Tool call: {tool_name}")
            print(f"  Arguments: {json.dumps(tool_args, indent=4)}")

            result = await call_remote_tool(tool_name, tool_args)

            content_text = ""
            for item in result.content:
                if hasattr(item, 'text'):
                    content_text += item.text

            print(f"  Result: {content_text[:200]}...\n")

            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_block.id,
                "content": content_text
            })

        # Feed tool results back to Claude
        messages.append({"role": "user", "content": tool_results})

        response = anthropic_client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            tools=claude_tools,
            messages=messages
        )

    for block in response.content:
        if hasattr(block, 'text'):
            print(block.text)

await ask_claude_with_mcp(
    "Which asset performed better over the last 24 hours, BTC or ETH?"
)